Base에서 실행

# Textcuboid


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## 1) Dataload Reuters  data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import reuters

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(test_split=0.2)

category_list = []
for i in range(46):
    category_list.append(0)
print(len(category_list))

for i in train_labels:
    category_list[i] += 1

extracted_index = []

for i, v in enumerate(category_list):
    if v >= 100:
#         print(i, v)
        extracted_index.append(i)


extracted_train_data = []
for i, v in enumerate(train_labels):
    if v in extracted_index:
        extracted_train_data.append(train_data[i])

extracted_test_data = []
for i, v in enumerate(test_labels):
    if v in extracted_index:
        extracted_test_data.append(test_data[i])

extracted_train_labels = []
for i, v in enumerate(train_labels):
    if v in extracted_index:
        extracted_train_labels.append(train_labels[i])

extracted_test_labels = []
for i, v in enumerate(test_labels):
    if v in extracted_index:
        extracted_test_labels.append(test_labels[i])

word_to_index = reuters.get_word_index()

index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value+3] = key # pad, sos, unk, NULL 포함

for index, token in enumerate(("<pad>", "<sos>", "<unk>")): # index=3은 NULL
    index_to_word[index] = token

#문장 추출
to_txt=[]
for i in range(7828):
  
    to_txt.append(' '.join([index_to_word[index] for index in extracted_train_data[i]])[6:])
    
for i in range(1934):
    to_txt.append(' '.join([index_to_word[index] for index in extracted_test_data[i]])[6:])

46


In [3]:
y=extracted_train_labels+extracted_test_labels

encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=label[:7828]
y_test=label[7828:]

In [4]:
# 방법2) 케라스 내장함수 활용
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(y_train)
one_hot_test_labels = to_categorical(y_test)

In [5]:
x_train=to_txt[:7828]
x_test=to_txt[7828:]

sos_x_train=[]
sos_x_test=[]
for sen in x_train:
    sos_x_train.append('<sos> '+sen)
for sen in x_test:
    sos_x_test.append('<sos> '+sen)

all_txt=sos_x_train+sos_x_test

from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_txt)

vocab_size =len(tokenizer.word_index)+1 #1을 더해야 에러가 안터짐 토큰 영향으로 보임

x_train_encoded = tokenizer.texts_to_sequences(sos_x_train)
x_test_encoded = tokenizer.texts_to_sequences(sos_x_test)

max_len = 300

xtext_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_encoded, maxlen=max_len)
xtext_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_encoded, maxlen=max_len)

## 2) 어텐션 레이어 정의

In [6]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs


class TransformerBlock(tf.keras.layers.Layer):
    
        
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.dff = dff
        self.rate = rate
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(dff, activation="relu"),
             tf.keras.layers.Dense(embedding_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs) # 첫번째 서브층 : 멀티 헤드 어텐션
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output) # Add & Norm
        ffn_output = self.ffn(out1) # 두번째 서브층 : 포지션 와이즈 피드 포워드 신경망
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output) # Add & Norm
    
    def get_config(self):
        config = super(TransformerBlock, self).get_config()
        config.update({
            'embedding_dim': self.embedding_dim,
            'num_heads': self.num_heads,
            'dff': self.dff,
            'rate': self.rate
        })
        return config


class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embedding_dim, **kwargs):
        super(TokenAndPositionEmbedding, self).__init__(**kwargs)
    
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=max_len, output_dim=embedding_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
    def get_config(self):
        config = super(TokenAndPositionEmbedding, self).get_config()
        config.update({
            'max_len': self.max_len,
            'vocab_size': self.vocab_size,
            'embedding_dim': self.embedding_dim
        })
        return config

## 3) textcuboid 분류

In [7]:
textcuboid=np.load('./1-Channel textcuboid_reuters(bert).npy')

In [8]:
textcuboid_test=np.load('./1-Channel textcuboid_test_reuters(bert).npy')

In [9]:
import keras
from tensorflow.keras import layers

In [19]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, verbose=1),
]

In [20]:
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers

embedding_dim = 1024  # 각 단어의 임베딩 벡터의 차원   #2048  #1024
num_heads = 1  # 어텐션 헤드의 수
dff = 1024  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기  #128  #1024
 
Text_inputs = Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
Text_x = embedding_layer(Text_inputs)
transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
Text_x = transformer_block(Text_x)
Text_x = transformer_block(Text_x)
Text_x = tf.keras.layers.GlobalAveragePooling1D()(Text_x)
Text_x = tf.keras.layers.Dropout(0.5)(Text_x)
Text_x = tf.keras.layers.Dense(1024, activation="relu")(Text_x)

BERT_input = Input(shape=(119, 768))
conv1 = Conv1D(1024, 1, padding='valid', activation='relu')(BERT_input)
pooling = GlobalMaxPooling1D()(conv1)
BERT_x = Dense(1024, activation='relu')(pooling)

x = layers.concatenate([BERT_x, Text_x], axis=-1)
x = Dropout(0.5)(x)

output_layer = Dense(12, activation='softmax')(x)

model = Model(inputs=[BERT_input, Text_inputs], outputs=output_layer)

In [21]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=3, min_lr=0.0001
    ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, verbose=1),
]

In [22]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 token_and_position_embedding_1  (None, 300, 1024)   29427712    ['input_3[0][0]']                
  (TokenAndPositionEmbedding)                                                                     
                                                                                                  
 transformer_block_1 (Transform  (None, 300, 1024)   6301696     ['token_and_position_embedding_1[
 erBlock)                                                        0][0]',                          
                                                                  'transformer_block_1[0][0]

In [24]:
textcuboid_train=textcuboid[1000:]
text_train=xtext_train[1000:]
textcuboid__val=textcuboid[:1000]
text_val=xtext_train[:1000]
y_train=one_hot_train_labels[1000:]
y_val=one_hot_train_labels[:1000]

text_test=xtext_test
y_test=one_hot_test_labels

In [25]:
history = model.fit([textcuboid_train,text_train], y_train,callbacks=callbacks, epochs=15,batch_size=256,validation_data=([textcuboid__val,text_val], y_val))

Epoch 1/15
27/27 [==============================] - 14s 460ms/step - loss: 5.8246 - accuracy: 0.5101 - val_loss: 0.9950 - val_accuracy: 0.7170 - lr: 0.0010
Epoch 2/15
27/27 [==============================] - 12s 438ms/step - loss: 0.8767 - accuracy: 0.7579 - val_loss: 0.6835 - val_accuracy: 0.8050 - lr: 0.0010
Epoch 3/15
27/27 [==============================] - 12s 440ms/step - loss: 0.6304 - accuracy: 0.8264 - val_loss: 0.5826 - val_accuracy: 0.8430 - lr: 0.0010
Epoch 4/15
27/27 [==============================] - 12s 440ms/step - loss: 0.4959 - accuracy: 0.8720 - val_loss: 0.5354 - val_accuracy: 0.8630 - lr: 0.0010
Epoch 5/15
27/27 [==============================] - 12s 440ms/step - loss: 0.4067 - accuracy: 0.8985 - val_loss: 0.4960 - val_accuracy: 0.8740 - lr: 0.0010
Epoch 6/15
27/27 [==============================] - 12s 441ms/step - loss: 0.3244 - accuracy: 0.9174 - val_loss: 0.4893 - val_accuracy: 0.8800 - lr: 0.0010
Epoch 7/15
27/27 [==============================] - 12s 449ms/st

In [26]:
model.evaluate([textcuboid_test,text_test],y_test)

61/61 [==============================] - 1s 20ms/step - loss: 0.5785 - accuracy: 0.8785


[0.5784855484962463, 0.878490149974823]

In [27]:
from keras.models import load_model
custom_objects = {"TokenAndPositionEmbedding": TokenAndPositionEmbedding, "TransformerBlock": TransformerBlock}
model = load_model('best_model.h5', custom_objects=custom_objects)
model.evaluate([textcuboid_test,text_test],y_test)

61/61 [==============================] - 1s 20ms/step - loss: 0.5257 - accuracy: 0.8687


[0.5257241725921631, 0.8686659932136536]

In [46]:
#은닉층 조절
np.average([88.1,87.7,87.6])

87.8